In [ ]:
import pandas as pd
import numpy as np
import random
import math
from typing import Dict, List, Tuple
from plotly.subplots import make_subplots

import plotly.express as px
pd.options.plotting.backend = "plotly"

In [ ]:
# Market behavior simulation functions
def short_sin(day:int, cycle:int):
    value = 1.5 + (0.5 * math.sin((day + 1.5 * cycle) / (cycle / (2*math.pi))))
    return value


def short_cos(day:int, cycle:int):
    value = 1.55 + (0.5 * math.cos((day + 0.5 * cycle) / (cycle / (2*math.pi))))
    return value


def long_sin(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi)))) * (10 - (day / cycle)) / 10
    #value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


def long_cos(day:int, cycle:int, offset:float, amplitude:float):
    value = amplitude * (1 + (0.5 * math.cos(((day + 2 * cycle * offset) % (2 * cycle)) / (cycle / math.pi))) * (10 - (day / (2 * cycle))) / 10 )
    #value = 1 + (0.5 * math.cos(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


# Reward rate framework
def rr_framework(supply:int):
    if supply > 10000000000000:
      return 0.0000625
    elif supply > 1000000000000:
      return 0.000125
    elif supply > 100000000000:
      return 0.00025
    elif supply > 10000000000:
      return 0.0005
    elif supply > 1000000000:
      return 0.001
    elif supply > 1000000000:
      return 0.001
    elif supply > 100000000:
      return 0.002
    else:
      return 0.004

In [ ]:
class ModelParams():
    def __init__(self, seed:int, inflow_type:str, horizon:int, ask_factor:float, bid_factor:float, cushion_factor:float, target_ma:float, lower_wall:float, upper_wall:float, lower_cushion:float, upper_cushion:float, reinstate_window:int, min_counter_reinstate:int, min_premium_target:int, max_outflow_rate:float, supply_amplitude:int, reserve_change_speed:float, max_liq_ratio:float, cycle_reweights:float, release_capture:float, demand_factor:float, supply_factor:float, initial_supply:float, initial_reserves:float, initial_liq_usd:float, arb_factor:float, initial_price:float, initial_target:float, target_price_function:str, short_cycle:int, long_cycle:int, long_sin_offset:float, long_cos_offset:float):
        self.seed = seed
        self.horizon = horizon
        self.cycle_reweights = cycle_reweights
        self.reserve_change_speed = reserve_change_speed
        self.short_cycle = short_cycle
        self.long_cycle = long_cycle
        self.long_sin_offset = long_sin_offset
        self.long_cos_offset = long_cos_offset
        self.supply_amplitude = supply_amplitude

        self.max_liq_ratio = max_liq_ratio
        self.min_premium_target = min_premium_target
        self.release_capture = release_capture
        self.max_outflow_rate = max_outflow_rate
        self.demand_factor = demand_factor
        self.supply_factor = supply_factor

        self.target_ma = target_ma
        self.lower_wall = lower_wall
        self.upper_wall = upper_wall
        self.lower_cushion = lower_cushion
        self.upper_cushion = upper_cushion
        self.bid_factor = bid_factor
        self.ask_factor = ask_factor
        self.cushion_factor = cushion_factor
        self.reinstate_window = reinstate_window
        self.min_counter_reinstate = min_counter_reinstate

        self.initial_supply = initial_supply
        self.initial_reserves = initial_reserves
        self.initial_liq_usd = initial_liq_usd
        self.initial_price = initial_price
        self.initial_target = initial_target
        self.target_price_function = target_price_function
        self.inflow_type = inflow_type
        self.arb_factor = arb_factor


In [ ]:
class Day():
    def __init__(self, params:ModelParams, prev_arbs:Dict[int, Tuple[float, float]], prev_lags=Dict[int, Tuple[int, Dict[int, float]]], prev_day=None):

        if prev_day is None:
            self.day = 1
            self.supply = params.initial_supply
            self.reward_rate = rr_framework(self.supply)
            self.price = params.initial_price
            self.liq_usd = params.initial_liq_usd
            self.liq_ohm = self.liq_usd / self.price
            self.k = (self.liq_usd ** 2) / self.price

            self.reserves_in = 0
            self.reserves_out = 0
            self.release_capture = 0
            self.ohm_traded = 0
            self.cum_ohm_purchased = 0
            self.cum_ohm_burnt = 0
            self.reserves = params.initial_reserves
            self.prev_reserves = params.initial_reserves

            self.ma_target = params.initial_target
            self.lower_target_wall = self.ma_target * (1 - params.lower_wall)
            self.lower_spread_wall = self.ma_target - self.lower_target_wall
            self.upper_target_wall = self.ma_target * (1 + params.upper_wall)
            self.upper_spread_wall = self.upper_target_wall - self.ma_target
            self.lower_target_cushion = self.ma_target * (1 - params.lower_cushion)
            self.lower_spread_cushion = self.ma_target - self.lower_target_cushion
            self.upper_target_cushion = self.ma_target * (1 + params.upper_cushion)
            self.upper_spread_cushion = self.upper_target_cushion - self.ma_target

            self.bid_capacity_target = params.bid_factor * self.reserves
            self.ask_capacity_target = params.ask_factor * self.reserves / self.upper_target_wall * (1 + self.lower_spread_wall + self.upper_spread_wall)
            self.bid_capacity_target_cushion = self.bid_capacity_target * params.cushion_factor
            self.ask_capacity_target_cushion = self.ask_capacity_target * params.cushion_factor
            self.bid_capacity = self.bid_capacity_target
            self.ask_capacity = self.ask_capacity_target
            self.bid_capacity_cushion = self.bid_capacity_target_cushion
            self.ask_capacity_cushion = self.ask_capacity_target_cushion
            
            self.prev_price = self.price
            self.prev_lower_target_wall = self.lower_target_wall
            self.prev_upper_target_wall = self.upper_target_wall
            
            self.market_demand = params.demand_factor
            self.market_supply = params.supply_factor
            self.arb_factor = params.arb_factor
            self.arb_demand = 0
            self.arb_supply = 0
            self.unwind_demand = 0
            self.unwind_supply = 0
            self.net_flow = random.uniform(self.liq_usd * self.market_supply, self.liq_usd * self.market_demand)
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)
            
            self.bid_counter = [0] * (params.reinstate_window - params.min_counter_reinstate) + [1] * params.min_counter_reinstate
            self.ask_counter = [0] * (params.reinstate_window - params.min_counter_reinstate) + [1] * params.min_counter_reinstate
            
        else:
            self.day = prev_day.day + 1
            self.reward_rate = rr_framework(prev_day.supply)
            self.supply = prev_day.supply * (1 + self.reward_rate) + min(prev_day.ohm_traded, 0)
            self.ma_target = calc_price_target(params=params, prev_day=prev_day, prev_lags=prev_lags)
            self.prev_price = prev_day.price

            # Walls
            self.lower_target_wall = self.ma_target * (1 - params.lower_wall)
            self.lower_spread_wall = self.ma_target - self.lower_target_wall
            self.upper_target_wall = self.ma_target * (1 + params.upper_wall)
            self.upper_spread_wall = self.upper_target_wall - self.ma_target

            # Cushions
            self.lower_target_cushion = self.ma_target * (1 - params.lower_cushion)
            self.lower_spread_cushion = self.ma_target - self.lower_target_cushion
            self.upper_target_cushion = self.ma_target * (1 + params.upper_cushion)
            self.upper_spread_cushion = self.upper_target_cushion - self.ma_target

            # Inside the range counters
            if prev_day.price > prev_day.lower_target_cushion:
                self.bid_counter = prev_day.bid_counter[1:] + [1]
            else:
                self.bid_counter = prev_day.bid_counter[1:] + [0]

            if prev_day.price < prev_day.upper_target_cushion:
                self.ask_counter = prev_day.ask_counter[1:] + [1]
            else:
                self.ask_counter = prev_day.ask_counter[1:] + [0]

            # Target capacities
            self.bid_capacity_target = params.bid_factor * prev_day.reserves
            self.ask_capacity_target = params.ask_factor * prev_day.reserves * (1 + prev_day.lower_spread_wall + prev_day.upper_spread_wall) / prev_day.upper_target_wall
            self.bid_capacity_target_cushion = self.bid_capacity_target * params.cushion_factor
            self.ask_capacity_target_cushion = self.ask_capacity_target * params.cushion_factor


            # Market dynamics
            self.net_flow = random.uniform(prev_day.treasury * prev_day.total_supply, prev_day.treasury * prev_day.total_demand) + prev_day.release_capture

            if params.inflow_type == 'historic':
                #load net inflows from CSV and use historical data
                a = 1
            elif params.inflow_type == 'random':
                self.market_demand = params.demand_factor * random.uniform(0.5, 3)
                self.market_supply = params.supply_factor * random.uniform(0.5, 3)
            else:
                self.market_demand = params.demand_factor * short_sin(self.day, params.short_cycle) * long_sin(self.day, params.long_cycle, params.long_sin_offset)
                self.market_supply = params.supply_factor * short_cos(self.day, params.short_cycle) * long_cos(self.day, params.long_cycle, params.long_cos_offset, params.supply_amplitude)


            # AMM k
            if prev_day.fmcap_treasury_ratio > params.min_premium_target:
                self.k = prev_day.k * (1 + self.reward_rate)**2
            else:
                self.k = prev_day.k
            
            
            # Reserve Intake
            if prev_day.reserves * (1 - params.max_liq_ratio) < prev_day.liq_usd * params.max_liq_ratio:
                self.reserves_in = (prev_day.liq_usd * params.max_liq_ratio - prev_day.reserves * (1 - params.max_liq_ratio)) / (params.reserve_change_speed * params.short_cycle)
            else:
                self.reserves_in = -2 * (prev_day.reserves * params.max_liq_ratio - prev_day.liq_usd * (1 - params.max_liq_ratio)) / (params.reserve_change_speed * params.short_cycle)
            
            if self.reserves_in < (-1) * prev_day.reserves:  # Ensure that the reserve release is limited by the total reserves left
                self.reserves = (-1) * prev_day.reserves                
            if self.reserves_in < (-1) * prev_day.reserves * params.max_outflow_rate:  # Ensure that the reserve release is limited by the max_outflow_rate
                self.reserves_in = (-1) * prev_day.reserves * params.max_outflow_rate

            natural_price = ((self.net_flow - self.reserves_in + prev_day.liq_usd) ** 2) / self.k

            # Real Bid Capacity - Cushion
            if sum(self.bid_counter) >= params.min_counter_reinstate and natural_price > self.lower_target_cushion:
                self.bid_capacity_cushion = self.bid_capacity_target_cushion
            elif natural_price < self.lower_target_cushion and natural_price >= self.lower_target_wall:
                self.bid_capacity_cushion = prev_day.bid_capacity_cushion + self.net_flow - self.reserves_in + prev_day.liq_usd - (self.k * self.lower_target_cushion) ** (1/2)
            else:
                self.bid_capacity_cushion = prev_day.bid_capacity_cushion
            
            if self.bid_capacity_cushion < 0:
                self.bid_capacity_cushion = 0
            elif self.bid_capacity_cushion > self.bid_capacity_target_cushion:
                self.bid_capacity_cushion = self.bid_capacity_target_cushion

            # Effective Bid Capacity Changes - Cushion
            if natural_price <= self.lower_target_cushion and natural_price > self.lower_target_wall:
                self.bid_change_cushion_ohm = (prev_day.bid_capacity_cushion - self.bid_capacity_cushion) / self.lower_target_cushion
                self.bid_change_cushion_usd = prev_day.bid_capacity_cushion - self.bid_capacity_cushion
            else:
                self.bid_change_cushion_ohm = 0
                self.bid_change_cushion_usd = 0

            if self.bid_change_cushion_ohm > prev_day.bid_capacity_cushion:  # ensure that change is smaller than capacity left
                self.bid_change_cushion_ohm = prev_day.bid_capacity_cushion / self.lower_target_cushion
                self.bid_change_cushion_usd = prev_day.bid_capacity_cushion

            # Real Bid Capacity - Totals
            if sum(self.bid_counter) >= params.min_counter_reinstate and natural_price > self.lower_target_cushion:
                self.bid_capacity = self.bid_capacity_target
            elif natural_price < self.lower_target_wall:
                self.bid_capacity = prev_day.bid_capacity + self.net_flow - self.reserves_in + prev_day.liq_usd - (self.k * self.lower_target_wall) ** (1/2)
            else:
                self.bid_capacity = prev_day.bid_capacity - self.bid_change_cushion_usd # update capacity total to account for the cushion

            if self.bid_capacity < 0:
                self.bid_capacity = 0
            elif self.bid_capacity > self.bid_capacity_target:
                self.bid_capacity = self.bid_capacity_target

            if self.bid_capacity_cushion > self.bid_capacity:
                self.bid_capacity_cushion = self.bid_capacity

            # Effective Bid Capacity Changes - Totals
            if natural_price >= self.lower_target_wall: #if wall wasn't used, update with cushion
                self.bid_change_ohm = self.bid_change_cushion_ohm
                self.bid_change_usd = self.bid_change_cushion_usd
            else:
                self.bid_change_ohm = self.bid_change_cushion_ohm + (prev_day.bid_capacity - self.bid_capacity - self.bid_change_cushion_usd) / self.lower_target_wall
                self.bid_change_usd = prev_day.bid_capacity - self.bid_capacity

            if self.bid_change_usd > prev_day.bid_capacity:  # ensure that change is smaller than capacity left
                self.bid_change_ohm = prev_day.bid_capacity / self.lower_target_wall
                self.bid_change_usd = prev_day.bid_capacity


            # Real Ask Capacity - Cushion
            if sum(self.ask_counter) >= params.min_counter_reinstate and natural_price < self.upper_target_cushion:
                self.ask_capacity_cushion = self.ask_capacity_target_cushion
            elif natural_price > self.upper_target_cushion and natural_price <= self.upper_target_wall:
                self.ask_capacity_cushion = prev_day.ask_capacity_cushion - (self.net_flow - self.reserves_in + prev_day.liq_usd) / self.upper_target_cushion + (self.k / self.upper_target_cushion) ** (1/2)
            else:
                self.ask_capacity_cushion = prev_day.ask_capacity_cushion

            if self.ask_capacity_cushion < 0:
                self.ask_capacity_cushion = 0
            elif self.ask_capacity_cushion > self.ask_capacity_target_cushion:
                self.ask_capacity_cushion = self.ask_capacity_target_cushion

             # Effective Ask Capacity Changes - Cushion
            if natural_price > self.upper_target_cushion and natural_price <= self.upper_target_wall:
                self.ask_change_cushion_ohm = prev_day.ask_capacity_cushion - self.ask_capacity_cushion
                self.ask_change_cushion_usd = self.upper_target_cushion * (prev_day.ask_capacity_cushion - self.ask_capacity_cushion)
            else:
                self.ask_change_cushion_ohm = 0
                self.ask_change_cushion_usd = 0
            
            if self.ask_change_cushion_ohm > prev_day.ask_capacity_cushion:  # ensure that change is smaller than capacity left
                self.ask_change_cushion_ohm = prev_day.ask_capacity_cushion
                self.ask_change_cushion_usd = prev_day.ask_capacity_cushion * self.upper_target_cushion
                            
            # Real Ask Capacity - Totals
            if sum(self.ask_counter) >= params.min_counter_reinstate and natural_price < self.upper_target_cushion:
                self.ask_capacity = self.ask_capacity_target
            elif natural_price > self.upper_target_wall:
                self.ask_capacity = prev_day.ask_capacity - (self.net_flow - self.reserves_in + prev_day.liq_usd) / self.upper_target_wall + (self.k / self.upper_target_wall) ** (1/2)
            else:
                self.ask_capacity = prev_day.ask_capacity - self.ask_change_cushion_ohm # update capacity total to account for the cushion

            if self.ask_capacity < 0:
                self.ask_capacity = 0
            elif self.ask_capacity > self.ask_capacity_target:
                self.ask_capacity = self.ask_capacity_target

            if self.ask_capacity_cushion > self.ask_capacity:
                self.ask_capacity_cushion = self.ask_capacity

            # Effective Ask Capacity Changes - Totals
            if natural_price <= self.upper_target_wall: #if wall wasn't used, update with cushion
                self.ask_change_ohm = self.ask_change_cushion_ohm
                self.ask_change_usd = self.ask_change_cushion_usd
            else:
                self.ask_change_ohm = prev_day.ask_capacity - self.ask_capacity
                self.ask_change_usd = self.ask_change_cushion_usd + (prev_day.ask_capacity - self.ask_capacity - self.ask_change_cushion_ohm) * self.upper_target_wall
            
            if self.ask_change_ohm > prev_day.ask_capacity:  # ensure that change is smaller than capacity left
                self.ask_change_ohm = prev_day.ask_capacity
                self.ask_change_usd = prev_day.ask_capacity * self.upper_target_wall
        

            # Liquidity and Reserves
            self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_in + self.bid_change_usd - self.ask_change_usd
            self.liq_ohm = self.k / self.liq_usd
            self.price = self.liq_usd / self.liq_ohm

            self.reserves_out = self.liq_usd - prev_day.liq_usd - self.net_flow
            self.reserves = max(prev_day.reserves - self.reserves_out, 0)
            self.prev_reserves = prev_day.reserves

            self.ohm_traded = (-2) * self.reserves_out / (self.price + prev_day.price)
            self.cum_ohm_purchased = prev_day.cum_ohm_purchased - self.ohm_traded
            self.cum_ohm_burnt = prev_day.cum_ohm_burnt - min(self.ohm_traded, 0)


            #still necessary? set it to zero, but need to double check with Zeus (cause he maintained it on his model)
            # Release Capture
            if self.day % params.short_cycle == 0:
                self.release_capture = (-1) * self.reserves_out * params.release_capture
            else:
                self.release_capture = 0

        
        self.net_flow_and_bond = self.net_flow - self.reserves_in
        self.treasury = self.liq_usd + self.reserves
        self.mcap = self.supply * self.price
        self.floating_supply = self.supply - self.liq_ohm
        self.floating_mcap = self.floating_supply * self.price
        self.liq_ratio = self.liq_usd / self.treasury
        self.reserves_ratio = self.reserves / self.liq_usd
        self.fmcap_treasury_ratio = self.floating_mcap / self.treasury
        self.liq_fmcap_ratio = self.liq_usd / self.floating_mcap
        self.total_demand = self.market_demand
        self.total_supply = self.market_supply
        self.total_net = self.total_demand + self.total_supply

        self.control_ask = sum(self.ask_counter)
        self.control_bid = sum(self.bid_counter)

        # Only for reporting purposes (to check calculations)
        prev_lags['price'][1][self.day] = self.price
        prev_lags['target'][1][self.day] = self.ma_target


In [ ]:
# Target price controller
def calc_price_target(params:ModelParams, prev_day:Day, prev_lags:Dict[int, Tuple[int, Dict[int, float]]]):
    if params.target_price_function == 'avg_lags':
        if prev_day.day % (params.short_cycle) == 0:
            s = 0
            lag_keys = set(prev_lags.keys()) - set(['price', 'target', 'natural', 'avg'])
            for key in lag_keys:
                days = prev_lags[key][1].keys()
                s += prev_lags[key][1][max(days)]
            avg_lag = s / len(lag_keys)
            return (prev_day.natural_target + avg_lag) / 2
        else:
            return prev_day.ma_target

    elif params.target_price_function == 'price_cycle_avg':
        if prev_day.day % (params.short_cycle) == 0:
            s = 0
            days = len(prev_lags['price'][1]) - 1
            days_reweight = int(params.short_cycle)
            if days > params.short_cycle:
                for i in range(days - days_reweight, days):
                    s += prev_lags['price'][1][i]
                return s / days_reweight
            else:
                 return prev_day.ma_target
        else:
            return prev_day.ma_target
            
    elif params.target_price_function == 'price_moving_avg':
        s = 0
        days = len(prev_lags['price'][1]) - 1
        days_ma = int(params.target_ma)
        if days > params.target_ma:
            for i in range(days - days_ma, days):
                s += prev_lags['price'][1][i]
            return s / days_ma
        elif days > 5:
            for i in range (1, days):
                s += prev_lags['price'][1][i]
            return s / days
        else:
            return prev_day.ma_target


def calc_natural_target(params:ModelParams, prev_day:Day):
    if prev_day.day % (params.short_cycle) == 0:
      if prev_day.day % (params.short_cycle * 4) == 0:
        return ((prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves) + prev_day.ma_target) / 2
      else:
        return prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves
    else:
        return prev_day.natural_target


def calc_lag(day:int, params:ModelParams, prev_lags:Dict[int, Tuple[int, Dict[int, float]]], num_days:int=3):
    for key, values in prev_lags.items():
        if key not in ('price', 'target', 'natural', 'avg'):
          lag_days = values[0]
          if day > lag_days:
              if key == 'lag1':
                  if day > params.short_cycle:
                      s = prev_lags['price'][1][day-1]
                      for i in range(1, num_days):
                          s += prev_lags['price'][1][day - (i * lag_days)]
                      prev_lags[key][1][day] = s / num_days
                  else:
                    prev_lags[key][1][day] = values[1][day - 1]
              else:
                prev_lags[key][1][day] = prev_lags['lag1'][1][day - values[0]]
          else:
            prev_lags[key][1][day] = values[1][day - 1]


### __SET SCENARIO PARAMETERS__

In [ ]:
# Simulate scenario with market operations
params = ModelParams(seed = 33 # seed number so all the simulations use the same randomness
    ,horizon = 360  # simulation timespan.
    ,short_cycle = 30  # short market cycle duration.
    ,cycle_reweights = 1  # reweights per short market cycle.
    ,long_cycle = 730  # long market cycle duration.
    ,long_sin_offset = 2  # demand function offset.
    ,long_cos_offset = 0  # supply function offset.
    ,supply_amplitude=0.8  # supply function amplitude.
    
    # Initial Parameters
    ,initial_supply = 25000000, initial_reserves = 340000000, initial_liq_usd = 70000000, initial_price = 30, initial_target = 30, target_price_function = 'price_moving_avg'

    ,inflow_type = 'cycles' # determines the inflow types. Either 'historical', 'random', or 'cycles' (sin/cos waves)
    ,demand_factor = 0.008  # % of OHM supply expected to drive market demand.
    ,supply_factor = -0.008  # % of OHM supply expected to drive market sell preasure.
    ,arb_factor = 0  # initial arb factor
    ,release_capture = 0  # % of reweight taken immediately by the market. --> I think it doesn't make sense anymore, that's why I set it to 0.

    ,max_liq_ratio = 0.33  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = 3  # minimum premium to keep adding liquidity as supply grows.
    ,max_outflow_rate = 0.0033 # max % of reservesUSD that can be released on a single day
    ,reserve_change_speed=1  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.

    ,ask_factor = 0.033  # % of floating supply that the treasury can deploy when price is trading above the upper target.
    ,bid_factor = 0.033  # % of the reserves that the treasury can deploy when price is trading below the lower target.
    ,cushion_factor = 0.33  # the percentage of a bid or ask to offer as a cushion.
    ,target_ma = 30  # length of the price target moving average (in days).
    ,lower_wall = 0.25  # determines lower wall price target at x% below the target price.
    ,upper_wall = 0.25  # determines upper wall price target at x% above the target price.
    ,lower_cushion = 0.2  # determines lower cushion price target at x% below the target price.
    ,upper_cushion = 0.2  # determines upper cushion price target at x% above the target price.
    ,reinstate_window = 7 # the window of time (in days) to reinstate a bid or ask.
    ,min_counter_reinstate = 6 # number of days within the reinstate window that conditions are true to reinstate a bid or ask.
)

lags = {
    'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target})
}

arbs = {}

random.seed(params.seed)
simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}
simulation_without = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}

for i in range (2, params.horizon):
    simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])

In [ ]:
# Simulate scenario with market operations - DO NOT MODIFY ANYTHING
params_without = ModelParams(seed = params.seed # seed number so all the simulations use the same randomness
    ,horizon = params.horizon  # simulation timespan.
    ,short_cycle = params.short_cycle  # short market cycle duration.
    ,cycle_reweights = params.cycle_reweights  # reweights per short market cycle.
    ,long_cycle = params.long_cycle  # long market cycle duration.
    ,long_sin_offset = params.long_sin_offset  # demand function offset.
    ,long_cos_offset = params.long_cos_offset  # supply function offset.
    ,supply_amplitude = params.supply_amplitude  # supply function amplitude.
    
    # Initial Parameters
    ,initial_supply = params.initial_supply, initial_reserves = params.initial_reserves, initial_liq_usd = params.initial_liq_usd, initial_price = params.initial_price, initial_target = params.initial_target, target_price_function = params.target_price_function

    ,inflow_type = params.inflow_type # determines the inflow types. Either 'historical', 'random', or 'cycles' (sin/cos waves)
    ,demand_factor = params.demand_factor  # % of OHM supply expected to drive market demand.
    ,supply_factor = params.supply_factor  # % of OHM supply expected to drive market sell preasure.
    ,arb_factor = params.arb_factor  # initial arb factor
    ,release_capture = params.release_capture  # % of reweight taken immediately by the market. --> I think it doesn't make sense anymore, that's why I set it to 0.

    ,max_liq_ratio = params.max_liq_ratio  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = params.min_premium_target  # minimum premium to keep adding liquidity as supply grows.
    ,max_outflow_rate = params.max_outflow_rate # max % of reservesUSD that can be released on a single day
    ,reserve_change_speed=params.reserve_change_speed  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.

    ,ask_factor = 0  # % of floating supply that the treasury can deploy when price is trading above the upper target.
    ,bid_factor = 0  # % of the reserves that the treasury can deploy when price is trading below the lower target.
    ,cushion_factor = 0  # the percentage of a bid or ask to offer as a cushion.
    ,target_ma = params.target_ma  # length of the price target moving average (in days).
    ,lower_wall = params.lower_wall  # determines lower wall price target at x% below the target price.
    ,upper_wall = params.upper_wall  # determines upper wall price target at x% above the target price.
    ,lower_cushion = params.lower_cushion  # determines lower cushion price target at x% below the target price.
    ,upper_cushion = params.upper_cushion  # determines upper cushion price target at x% above the target price.
    ,reinstate_window = params.reinstate_window # the window of time (in days) to reinstate a bid or ask.
    ,min_counter_reinstate = params.min_counter_reinstate # number of days within the reinstate window that conditions are true to reinstate a bid or ask.
)

lags_without = {
    'price': (0, {1: params_without.initial_price}), 'target': (0, {1: params_without.initial_target}), 'avg': (0, {1: params_without.initial_target})
}

arbs_without = {}

random.seed(params_without.seed)
simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without)}

for i in range (2, params_without.horizon):
    simulation_without[f'day{i}'] = Day(params=params_without, prev_arbs=arbs_without, prev_lags=lags_without, prev_day=simulation_without[f'day{i-1}'])

### __PLOT RESULTS__

In [ ]:
# Protocol variables
df1 = pd.DataFrame(columns = ['Type', 'NetFlow', 'Price', 'RealTarget', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumPurchasedOHM', 'CumBurntOHM']) 
for day, data in simulation.items():
    df1.loc[day] = ['withMarketOps', float(data.net_flow), float(data.price), float(data.ma_target), float(data.lower_target_cushion), float(data.upper_target_cushion), float(data.lower_target_wall), float(data.upper_target_wall), float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_purchased), float(data.cum_ohm_burnt)]

df2 = pd.DataFrame(columns = ['Type', 'NetFlow', 'Price', 'RealTarget', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumPurchasedOHM', 'CumBurntOHM']) 
for day, data in simulation_without.items():
        df2.loc[day] = ['withoutMarketOps', float(data.net_flow), float(data.price), float(data.ma_target), np.nan, np.nan, np.nan, np.nan, float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_purchased), float(data.cum_ohm_burnt)]
df=pd.concat([df1, df2])

# Guidance variables
guidance_df = pd.DataFrame(columns = ['BidCapacity', 'AskCapacity', 'BidCapacityCushion', 'AskCapacityCushion', 'BidCapacityTargetCushion', 'AskCapacityTargetCushion', 'BidCapacityTarget', 'AskCapacityTarget', 'TradedOHM', 'ReservesOUT', 'AskCount', 'BidCount']) 
for day, data in simulation.items():
    guidance_df.loc[day] = [data.bid_capacity, data.ask_capacity, data.bid_capacity_cushion, data.ask_capacity_cushion, data.bid_capacity_target_cushion, data.ask_capacity_target_cushion, data.bid_capacity_target, data.ask_capacity_target, data.ohm_traded, data.reserves_out, data.control_ask, data.control_bid]

# Market dynamics variables
#market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb'])
market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'NetTotal'])
for day, data in simulation.items():
    #market_df.loc[day] = [data.market_demand, data.market_supply, data.arb_factor, data.arb_demand, data.arb_supply, data.unwind_demand, data.unwind_supply, data.total_demand, data.total_supply, data.total_net, data.net_arb]
    market_df.loc[day] = [data.market_demand, data.market_supply, data.total_net]


In [ ]:
# Plot multivariable charts - Comparison vs without market operations

fig = df[['NetFlow', 'LiqUSD', 'Reserves', 'Treasury', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig.show()


fig1 = df[['MCap','FloatingMCap', 'LiqUSD', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig1.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig1.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig1.show()


fig2 = df[['Price','RealTarget', 'LowerTargetWall', 'UpperTargetWall', 'LowerTargetCushion', 'UpperTargetCushion', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig2.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig2.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig2.show()


fig3 = df[['Supply','FloatingSupply', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig3.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig3.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig3.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig3.show()


fig3b = df[['CumBurntOHM', 'CumPurchasedOHM', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig3b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig3b.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig3b.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig3b.show()


In [ ]:
# Plot multivariable charts

fig2b = df1[['Price','RealTarget', 'LowerTargetWall', 'UpperTargetWall', 'LowerTargetCushion', 'UpperTargetCushion']].plot()
fig2b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig2b.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig2b.show()


fig4 = guidance_df[['BidCapacity', 'BidCapacityTarget', 'BidCapacityCushion', 'BidCapacityTargetCushion']].plot()
fig4.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig4.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig4.show()


fig4b = guidance_df[['BidCount']].plot()
fig4b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig4b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    fig4b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#fig4b.show()


fig2b.show()
fig4 = guidance_df[['AskCapacity', 'AskCapacityTarget', 'AskCapacityCushion', 'AskCapacityTargetCushion']].plot()
fig4.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig4.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig4.show()

fig4b = guidance_df[['AskCount']].plot()
fig4b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig4b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    fig4b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#fig4b.show()

fig5 = market_df[['MarketDemand', 'MarketSupply']].plot()
fig5.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig5.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig5.show()

In [ ]:
# Plot batch1
for col in df1.columns:
    if col in ('NetFlow', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        if col == 'LiqRatio':
            fig.add_hline(y=params.max_liq_ratio, line_width=0.5, line_dash="dash", line_color="grey")
        if col == 'LiqRatioAdj':
            fig.add_hline(y=params.max_liq_ratio/(1-params.max_liq_ratio), line_width=0.5, line_dash="dash", line_color="grey")
        if col == 'FloatingMCTreasuryPremium':
            fig.add_hline(y=params.min_premium_target, line_width=1, line_dash="dash", line_color="grey")
        fig.show()

In [ ]:
# Plot batch2
for col in df1.columns:
    if col in ('Reserves', 'LiqUSD', 'LiqOHM', 'ReservesIN', 'ReservesOUT', 'Treasury'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        fig.show()

In [ ]:
# Plot batch3
for col in df1.columns:
    if col in ('Supply', 'FloatingSupply', 'CumPurchasedOHM'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        fig.show()

In [ ]:
# Plot batch4
#for col in ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb']:
#    fig = market_df.plot(x=df.index, y=market_df[col])
#    fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
#    for i in range (params.short_cycle, params.horizon, params.short_cycle):
#        fig.add_vline(x=i, line_width=0.5, line_color="white", layer='below')
#    fig.show()


In [ ]:
# Market cycles (sin and cos waves)
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 10*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset, params.supply_amplitude)]

df2.plot(y=df2.columns)